In [5]:
import binascii
import socket
import struct

def main():
    # Sample IPv4 hex dump (replace this with your actual hex dump)
    ipv4_hex_dump = "4500003f960200008011ac950a19408eac12015dc2000035002b23f3c7c80100000100000000000002626207737573746563680365647502636e0000010001"

    # Decode the hex dump into binary
    ipv4_packet_data = binascii.unhexlify(ipv4_hex_dump)

    # Extract IP and UDP header
    ip_header = ipv4_packet_data[:20]
    udp_header = ipv4_packet_data[20:28]
    data = ipv4_packet_data[28:]

    # Extract source and destination IP addresses
    src_ip = struct.unpack('!L', ip_header[12:16])[0]
    dst_ip = struct.unpack('!L', ip_header[16:20])[0]

#     print(src_ip)
#     print(dst_ip)

    # 将剩余数据转换为16位形式并相加
    sum_remaining_data = 0
    if len(data) % 2 != 0:
        data += b'\x00'
    for i in range(0, len(data), 2):
        sum_remaining_data += int.from_bytes(data[i:i+2], byteorder='big')
#         print(hex(int.from_bytes(data[i:i+2], byteorder='big')))

    # Extract UDP header fields
    src_port, dst_port, udp_len, checksum_field = struct.unpack('!HHHH', udp_header)
    
#     src_ip=0x7f000001
#     dst_ip=0x7f000001
#     udp_len=0x0009
#     src_port=0xca5a
#     dst_port=0x04b0
#     checksum_field=0x0000
    # Calculate checksum
#     print(hex(src_ip>>16),hex(src_ip & 0xFFFF),hex(dst_ip >> 16),hex(dst_ip & 0xFFFF),hex(udp_len))
    sum1 = (src_ip >> 16) + (src_ip & 0xFFFF) + (dst_ip >> 16) + (dst_ip & 0xFFFF) + 0x0011 + udp_len + src_port + dst_port + 0x0000 + udp_len + sum_remaining_data
    sum2 = (sum1 & 0xFFFF) + (sum1 >> 16)
    sum3 = (sum2 & 0xFFFF) + (sum2 >> 16)
    # 计算checksum
    checksum = ~sum3 & 0xFFFF


    # Print information
    print("Source IP:", socket.inet_ntoa(ip_header[12:16]))
    print("Destination IP:", socket.inet_ntoa(ip_header[16:20]))
    print("Source Port:", src_port)
    print("Destination Port:", dst_port)
    print("UDP Length:", udp_len)
    print("Checksum Field:", hex(checksum_field))
    print("Calculated Checksum:", hex(checksum))
    print("Checksum Match:", "Yes" if checksum == checksum_field else "No")
#     print("sum3",hex(sum3))

if __name__ == "__main__":
    main()


Source IP: 10.25.64.142
Destination IP: 172.18.1.93
Source Port: 49664
Destination Port: 53
UDP Length: 43
Checksum Field: 0x23f3
Calculated Checksum: 0x23f3
Checksum Match: Yes
sum3 0xdc0c


In [4]:
src_ip=0x7f000001